In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline

C:\Users\Yogesh Singh\.conda\envs\tensorflow\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
In C:\Users\Yogesh Singh\.conda\envs\tensorflow\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Yogesh Singh\.conda\envs\tensorflow\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Yogesh Singh\.conda\envs\tensorflow\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\Yoge

#### The dataset is stored in `datasets` folder along with a `exit_status_desc.png file` of it's description.

In [2]:
df = pd.read_csv('datasets/exit_status.csv')
df.head()

,id,gender,age,dependents,lifetime,phone_services,internet_services,online_streaming,multiple_connections,premium_plan,online_protect,contract_plan,ebill_services,default_payment,monthly_charges,issues,exit_status
0,1689,Male,30-45,Yes,7,No,3G,Major User,NaN,No,Yes,Month-to-month,No,Physical,58.85,2,No
1,794,Female,18-30,No,6,No,3G,Major User,NaN,No,No,Month-to-month,No,Auto-payment,45.00,6,No
2,4211,Male,>60,No,24,Yes,4G,Major User,Yes,No,Yes,Month-to-month,Yes,Auto-payment,102.95,22,Yes
3,3318,Male,18-30,No,10,No,3G,No,NaN,No,Yes,Month-to-month,No,Physical,29.50,28,Yes
4,5245,Female,30-45,Yes,70,Yes,NaN,NaN,No,NaN,NaN,Two year,Yes,Auto-payment,20.15,13,No


#### Missing Values??

In [3]:
df.isna().sum()

id                        0
gender                    0
age                       0
dependents                0
lifetime                  0
phone_services            0
internet_services       259
online_streaming        458
multiple_connections    241
premium_plan            458
online_protect          458
contract_plan             0
ebill_services            0
default_payment           0
monthly_charges           0
issues                    0
exit_status               0
dtype: int64

#### There are a few missing values.

In [4]:
df.shape

(2600, 17)

#### Out of 2600 rows, atmost 500 are missing i.e. around 1/5th of total. In this case, I'm going to drop the missing rows, but we have options of filling that missing data, but for now, we are going to stick to dropping the null values.

In [5]:
df.dropna(inplace = True)

In [6]:
df.shape

(1708, 17)

#### Encode the target variable `exit_status`.

In [7]:
label_map = {'Yes': 1, 'No': 0}

In [8]:
df['exit_status'] = df['exit_status'].map(label_map)

In [10]:
df.head()

,id,gender,age,dependents,lifetime,phone_services,internet_services,online_streaming,multiple_connections,premium_plan,online_protect,contract_plan,ebill_services,default_payment,monthly_charges,issues,exit_status
2,4211,Male,>60,No,24,Yes,4G,Major User,Yes,No,Yes,Month-to-month,Yes,Auto-payment,102.95,22,1
5,6446,Male,45-60,Yes,58,Yes,4G,No,Yes,Yes,No,One year,Yes,Auto-payment,89.85,10,0
6,4269,Male,30-45,No,17,Yes,4G,No,Yes,No,Yes,Month-to-month,No,Auto-payment,90.20,29,1
8,3385,Male,18-30,No,12,Yes,4G,Major User,Yes,No,No,Month-to-month,Yes,Online Transfer,96.05,20,1
9,2795,Female,>60,No,9,Yes,4G,No,Yes,No,No,Month-to-month,Yes,Online Transfer,74.55,28,1


#### Once that's done, let's seperate our target and feature variables.

In [11]:
x = df.drop(['id', 'exit_status'], axis = 1)
y = df.exit_status

In [12]:
x.head()

,gender,age,dependents,lifetime,phone_services,internet_services,online_streaming,multiple_connections,premium_plan,online_protect,contract_plan,ebill_services,default_payment,monthly_charges,issues
2,Male,>60,No,24,Yes,4G,Major User,Yes,No,Yes,Month-to-month,Yes,Auto-payment,102.95,22
5,Male,45-60,Yes,58,Yes,4G,No,Yes,Yes,No,One year,Yes,Auto-payment,89.85,10
6,Male,30-45,No,17,Yes,4G,No,Yes,No,Yes,Month-to-month,No,Auto-payment,90.20,29
8,Male,18-30,No,12,Yes,4G,Major User,Yes,No,No,Month-to-month,Yes,Online Transfer,96.05,20
9,Female,>60,No,9,Yes,4G,No,Yes,No,No,Month-to-month,Yes,Online Transfer,74.55,28


In [13]:
y.head()

2    1
5    0
6    1
8    1
9    1
Name: exit_status, dtype: int64

#### Let's split our data into train and test set

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

#### Encode categorical variables.

In [15]:
cat_features = x_train.columns[x_train.dtypes == object].tolist()

In [16]:
cat_features

['gender',
 'age',
 'dependents',
 'phone_services',
 'internet_services',
 'online_streaming',
 'multiple_connections',
 'premium_plan',
 'online_protect',
 'contract_plan',
 'ebill_services',
 'default_payment']

#### Time to create a column transformer for the encoding.

In [17]:
col_trans = make_column_transformer((OneHotEncoder(), cat_features), remainder = 'passthrough')

#### Our inputs are now ready and it's time to define our model. As usual, going with default parameters.

In [18]:
logistic_model = LogisticRegression()

#### Now, we create a pipeline to apply our column transformation and then pass that as input to our model. We can also apply various other transformations such as scaling, normalization etc. in our pipeline.

In [19]:
pipe = make_pipeline(col_trans, logistic_model)
pipe.fit(x_train, y_train)

C:\Users\Yogesh Singh\.conda\envs\tensorflow\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['gender', 'age',
                                                   'dependents',
                                                   'phone_services',
                                                   'internet_

In [20]:
y_pred = pipe.predict(x_test)

#### Let's have a look at our metrics.

In [21]:
print(accuracy_score(y_test, y_pred))

0.7446393762183235


In [22]:
print(confusion_matrix(y_test, y_pred))

[[132  95]
 [ 36 250]]


In [23]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.58      0.67       227
           1       0.72      0.87      0.79       286

    accuracy                           0.74       513
   macro avg       0.76      0.73      0.73       513
weighted avg       0.75      0.74      0.74       513



#### Well, ~74-75% accuracy ain't that bad and that too when we have done minimum preprocessing and default parameter were used. To improve accuracy, we can try scaling our data and normalizing it. Also, trying out other models such as Random Forest, Gradient Boosting etc. can be useful.